In [1]:
'''
written by Lorenz Muller
'''

import numpy as np
import tensorflow as tf
from time import time
import sys
from dataLoader import loadData
import os

In [2]:
seed = 47
np.random.seed(seed)

In [3]:
# # Downloading Movielens-1m
# !curl -O http://files.grouplens.org/datasets/movielens/ml-1m.zip
# #     http://www.grouplens.org/system/files/ml-1m.zip
# !unzip ml-1m.zip
# !cd ml-1m/

In [4]:
# import zipfile
# with zipfile.ZipFile('ml-1m.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

In [5]:
# load data
tr, vr = loadData('./ml-1m/ratings.dat', delimiter='::', seed=seed, transpose=False, valfrac=0.1)

reading data...
data read in 4.956859111785889 seconds
loaded dense data matrix


In [6]:
# np.random.seed(seed)
# path = './ml-1m/ratings.dat'
# delimiter='::'

# tic = time()
# print('reading data...')
# data = np.loadtxt(path, skiprows=0, delimiter=delimiter).astype('int32')
# print('data read in', time() - tic, 'seconds')

# n_u = np.unique(data[:, 0]).shape[0]  # number of users
# n_m = np.unique(data[:, 1]).shape[0]  # number of movies
# n_r = data.shape[0]  # number of ratings

# # these dictionaries define a mapping from user/movie id to to user/movie number (contiguous from zero)
# udict = {}
# for i, u in enumerate(np.unique(data[:, 0]).tolist()):
#     udict[u] = i
# mdict = {}
# for i, m in enumerate(np.unique(data[:, 1]).tolist()):
#     mdict[m] = i

# # shuffle indices
# idx = np.arange(n_r)
# np.random.shuffle(idx)

# trainRatings = np.zeros((n_u, n_m), dtype='float32')
# validRatings = np.zeros((n_u, n_m), dtype='float32')

In [7]:
tr

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
vr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
# import pandas as pd 

# ratings_list = [i.strip().split("::") for i in open('./ml-1m/ratings.dat', 'r').readlines()]
# users_list = [i.strip().split("::") for i in open('./ml-1m/users.dat', 'r').readlines()]
# movies_list = [i.strip().split("::") for i in open('./ml-1m/movies.dat', 'r').readlines()]

# ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
# movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
# movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [10]:
# R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
# R_df.head()

In [11]:
# ratings = np.array(R_df.values, dtype=int)
# n_users = ratings.shape[0]
# n_items = ratings.shape[1]

In [12]:
# t = tr + vr

In [13]:
(vr > 0).sum()

100021

In [14]:
tr.shape, vr.shape

((6040, 3706), (6040, 3706))

In [15]:
def get_sparsity(mat):
    sparsity = float(len(mat.nonzero()[0]))
    sparsity /= (mat.shape[0] * mat.shape[1])
    sparsity *= 100
    return sparsity

In [16]:
get_sparsity(tr)

4.021525859265269

In [17]:
get_sparsity(vr)

0.44683670296601535

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.init as init
from os.path import isfile, isdir, join
import os
# from tensorboard_logger import configure, log_value

In [19]:
lrD = 5e-4
lrG = 5e-4
batch_size = 100
cuda = True
epochs = 400 #change
device = 5
seed = 1
nz = 20
d_iter = 5
g_iter = 1
lamba = 1e-2 # constant for L2 penalty (diversity)
name = "mnist-experiment"
# configure("runs/run-" + args.name, flush_secs=5)
torch.manual_seed(seed)


# data_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#     transform=transforms.Compose([
#     transforms.ToTensor(),
#     ])), batch_size=batch_size, shuffle=True)

In [20]:
train = tr

In [21]:
features_length = train.shape[1]
class NetD(torch.nn.Module):
    def __init__(self, use_cuda=True):
        super(NetD, self).__init__()
        self.use_cuda = use_cuda
        # top
        self.t1 = torch.nn.Linear(features_length, 1024)
        # bottom
        self.b1 = torch.nn.Linear(features_length, 1024)
        # combined
        self.fc = torch.nn.Linear(2 * 1024, features_length)
    def forward(self, xr, xf):
        # get filt
#         filt = (torch.abs((real > 0.3).float() * fake - real))/real.shape[0]
        filt = torch.abs((real != 0).float().cuda() * fake.cuda() - real.cuda())/(real != 0).sum()
#         filt = torch.abs((xr != 0).int() * xf - xr)
#         filt = 1 - (xr * (xf >= 0.5).float()) - ((1-xr) * (xf < 0.5).float())
        # random swap
        idr = torch.multinomial(torch.Tensor([0.5,0.5]), xr.size(0), replacement=True)
        idrx = idr.float().unsqueeze(1).expand_as(xr)
        if self.use_cuda: 
            idrx = idrx.cuda()
        idrx = Variable(idrx)
        xt = xr * idrx + xf * (1 - idrx)
        xb = xr * (1 - idrx) + xf * idrx
        # top : real
        xt = F.relu(self.t1(xt))
        # bottom : fake
        xb = F.relu(self.b1(xb))
        # combined
        x = torch.cat((xt, xb), 1)
        x = torch.tanh(self.fc(x))
        # apply filter, aggregate
#         print(filt.type(), x.type())
        x = filt * x

        x = x.mean(dim = 1).squeeze()
        # use sign, because of swapping
        sgn = idr * 2 - 1
        if self.use_cuda: 
            sgn = sgn.cuda()
        sgn = Variable(sgn.float())
        x = sgn * x
        return x
        
# netG = torch.nn.Sequential(
#     torch.nn.Linear(nz, 1024),
#     torch.nn.ReLU(),
#     torch.nn.Linear(1024, features_length),
#     torch.nn.Sigmoid()*5
#     )

class NetG(nn.Module):
    
    def __init__(self):

        super(NetG, self).__init__()

        self.net = torch.nn.Sequential( 
                                torch.nn.Linear(nz, 1024), 
                                torch.nn.ReLU(), 
                                torch.nn.Linear(1024, 1024), 
                                torch.nn.Sigmoid(), 
                                nn.Dropout(0.5),
                                torch.nn.Linear(1024, features_length), 
                                torch.nn.Sigmoid(), 
                                nn.Dropout(0.6)
                                )

#         self.net = nn.Sequential(
#                                  nn.Linear(nz,1024),
# #                                  nn.Dropout(0.3)
#                                  nn.ReLU(),
#                                  nn.Linear(1024,2048),
#                                  nn.Sigmoid(),
#                                  nn.Dropout(0.3),
#                                  nn.Linear(2048,features_length),
# #                                  nn.Sigmoid()
#                                  nn.Dropout(0.5)
#                                     )
        
    def forward(self, x):
        x = self.net(x)
#         return x
        return x*5 # to get values in range [0,5]
    
# networks
netD = NetD(use_cuda=True)
netG = NetG()
print(netG)
print(netD)
optimizerG = optim.RMSprop(netG.parameters(), lr=lrG)
optimizerD = optim.RMSprop(netD.parameters(), lr=lrD)
one = torch.FloatTensor([1])
mone = one * -1

NetG(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Sigmoid()
    (4): Dropout(p=0.5)
    (5): Linear(in_features=1024, out_features=3706, bias=True)
    (6): Sigmoid()
    (7): Dropout(p=0.6)
  )
)
NetD(
  (t1): Linear(in_features=3706, out_features=1024, bias=True)
  (b1): Linear(in_features=3706, out_features=1024, bias=True)
  (fc): Linear(in_features=2048, out_features=3706, bias=True)
)


In [22]:
one = torch.FloatTensor([1])
mone = one * -1
cuda = True
if cuda is True:
    netD.cuda()
    netG.cuda()
    one, mone = one.cuda(), mone.cuda()

In [23]:
for p in netD.parameters(): # reset requires_grad
    p.requires_grad = True #
    
for p in netG.parameters(): # reset requires_grad
    p.requires_grad = True #

In [24]:
def getRealSample(length=6):
     return Variable(torch.IntTensor(np.random.choice([0, 1], size=(batch_size, length))))

In [25]:
def get_random_batch(mat, batch_size=16):
    rand_rows = np.random.randint(mat.shape[0], size=batch_size)
#     print(mat.shape, rand_rows)
#     print(mat[rand_rows].shape)
    return mat[rand_rows]

In [26]:
get_random_batch(train, batch_size=batch_size).shape

(100, 3706)

In [27]:
train = torch.autograd.Variable(torch.Tensor(train))

In [28]:
def d_my(x_r, x_g):
    return torch.sum(torch.abs((x_r != 0).float() * x_g - x_r))

In [29]:
train.shape

torch.Size([6040, 3706])

In [30]:
import pandas as pd 

ratings_list = [i.strip().split("::") for i in open('./ml-1m/ratings.dat', 'r').readlines()]
users_list = [i.strip().split("::") for i in open('./ml-1m/users.dat', 'r').readlines()]
movies_list = [i.strip().split("::") for i in open('./ml-1m/movies.dat', 'r').readlines()]

ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)

ratings = np.array(R_df.values, dtype=int)
n_users = ratings.shape[0]
n_items = ratings.shape[1]

In [31]:
ratings

array([[5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
get_sparsity(ratings)

4.468362562231285

In [33]:
get_sparsity(tr)

4.021525859265269

In [34]:
get_sparsity(vr)

0.44683670296601535

In [35]:
steps_per_epoch = 150
gen_iterations = 0
eval_losses = []
for epoch in range(epochs):
#     data_iter = iter(data_loader)
    i = 0
    while i < steps_per_epoch:
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update
        d_iter = d_iter
        j = 0
        while j < d_iter:
            j += 1
            # load real data
            i += 1
#             X, _ = data_iter.next()
            X = get_random_batch(train, batch_size=batch_size)
#             X += torch.Tensor(np.random.normal(0, 0.2, X.shape))
#             print(X >= 0.5)
# #             X = X.view(X.size(0), -1)
#             X = (X >= 0.5).float()
            if cuda: 
                X = X.cuda()
            real = Variable(X)
            # generate fake data
            noise = torch.randn(batch_size, nz)
            if cuda: 
                noise = noise.cuda()
            with torch.no_grad():
                noisev = Variable(noise) # totally freeze netG
            fake = Variable(netG(noisev).data)
#             print(real.shape, fake.shape)
    
            # compute gradient, take step
            netD.zero_grad()
#             print('real', real)
#             print('fake', fake[:,0].sum())
            out = netD(real, fake)
            
            outputD = torch.mean(out) + lamba * out.norm()
            stdD = torch.std(out)
            outputD.backward(mone)
            optimizerD.step()
#             print(out.shape)
        ############################
        # (2) Update G network
        ###########################
        g_iter = g_iter
        j = 0
        while j < g_iter:
            j += 1
            for p in netD.parameters():
                p.requires_grad = False # to avoid computation
            netG.zero_grad()
            
            # load real data
            i += 1
            X = get_random_batch(train, batch_size=batch_size)
#             X += torch.Tensor(np.random.normal(0, 0.2, X.shape))
#             X = X.view(X.size(0), -1)
#             X = (X >= 0.5).float()
            if cuda: 
                X = X.cuda()
            real = Variable(X)
            
            # update generator
            noise = torch.randn(batch_size, nz)
            if cuda: 
                noise = noise.cuda()
            noisev = Variable(noise)
            fake = netG(noisev)
            out = netD(real, fake)
            outputG = torch.mean(out) + lamba * out.norm()
            stdG = torch.std(out)
            outputG.backward(one)
            optimizerG.step()

            gen_iterations += 1

#             print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f '% (epoch, epochs, i, len(data_loader), gen_iterations, outputD.item(), outputG.item()))
#             print('output_D', outputD.item(), gen_iterations)
#             print('output_G', outputG.item(), gen_iterations)
#             print('std_D', stdD.item(), gen_iterations)
#             print('std_G', stdG.item(), gen_iterations)
            
            # evaluation
            if gen_iterations % 100 == 0: # todo- to change
#                 gen.eval()
#                 z_vector_eval = make_some_noise(128)
#                 fake_rows_eval = gen(z_vector_eval)
#                 real_rows_eval = get_random_batch(train, 128)
        #         print(fake_rows[0][:10]) enable to see some results
                eval_loss = F.mse_loss(fake, real, reduction='mean')
                eval_losses.append(eval_loss)
                print('Epoch number {}. my distance between random real and fake samples {}'.format(epoch, d_my(real, fake)))
                print('Epoch number {}. MSE distance between random real and fake samples {}'.format(epoch, eval_loss))

Epoch number 3. my distance between random real and fake samples 42916.7734375
Epoch number 3. MSE distance between random real and fake samples 10.16589069366455
Epoch number 7. my distance between random real and fake samples 34558.19140625
Epoch number 7. MSE distance between random real and fake samples 8.85556411743164
Epoch number 11. my distance between random real and fake samples 38496.1015625
Epoch number 11. MSE distance between random real and fake samples 8.10557746887207
Epoch number 15. my distance between random real and fake samples 36116.75
Epoch number 15. MSE distance between random real and fake samples 7.6949334144592285
Epoch number 19. my distance between random real and fake samples 28026.5546875
Epoch number 19. MSE distance between random real and fake samples 7.422409534454346
Epoch number 23. my distance between random real and fake samples 32105.15625
Epoch number 23. MSE distance between random real and fake samples 7.229992389678955
Epoch number 27. my d

Epoch number 199. my distance between random real and fake samples 40020.421875
Epoch number 199. MSE distance between random real and fake samples 5.817627429962158
Epoch number 203. my distance between random real and fake samples 34299.80078125
Epoch number 203. MSE distance between random real and fake samples 5.930644989013672
Epoch number 207. my distance between random real and fake samples 34953.1796875
Epoch number 207. MSE distance between random real and fake samples 5.935337066650391
Epoch number 211. my distance between random real and fake samples 36271.5546875
Epoch number 211. MSE distance between random real and fake samples 5.8728108406066895
Epoch number 215. my distance between random real and fake samples 30598.20703125
Epoch number 215. MSE distance between random real and fake samples 5.869164943695068
Epoch number 219. my distance between random real and fake samples 37352.13671875
Epoch number 219. MSE distance between random real and fake samples 5.86635303497

Epoch number 395. my distance between random real and fake samples 38255.19921875
Epoch number 395. MSE distance between random real and fake samples 5.610450744628906
Epoch number 399. my distance between random real and fake samples 31917.599609375
Epoch number 399. MSE distance between random real and fake samples 5.498556613922119


In [36]:
fake

tensor([[4.0677, 3.0749, 3.1720,  ..., 0.0000, 3.2269, 0.0000],
        [4.0740, 0.0000, 3.0261,  ..., 0.0000, 0.0000, 3.7588],
        [0.0000, 3.4719, 0.0000,  ..., 3.7493, 0.0000, 0.0000],
        ...,
        [4.1807, 3.0051, 0.0000,  ..., 4.5524, 3.6993, 0.0000],
        [0.0000, 3.0363, 0.0000,  ..., 0.0000, 0.0000, 3.1808],
        [0.0000, 2.8532, 0.0000,  ..., 0.0000, 0.0000, 3.5762]],
       device='cuda:0', grad_fn=<MulBackward0>)

In [37]:
eval_loss = [c.item() for c in eval_losses]

In [38]:
import matplotlib.pyplot as plt

plt.plot(eval_loss)
plt.show()

<Figure size 640x480 with 1 Axes>

In [39]:
noise = torch.randn(tr.shape[0], nz)
if cuda: 
    noise = noise.cuda()
noisev = Variable(noise)
fake = netG(noisev)

In [40]:
fake.shape

torch.Size([6040, 3706])

In [41]:
fake = fake.round()

In [42]:
fake.shape

torch.Size([6040, 3706])

In [43]:
print(5, (5 == fake.round()).sum(), (5 == (tr + vr).round()).sum())
print(4, (4 == fake.round()).sum(), (4 == (tr + vr).round()).sum())
print(3, (3 == fake.round()).sum(), (3 == (tr + vr).round()).sum())
print(2, (2 == fake.round()).sum(), (2 == (tr + vr).round()).sum())
print(1, (1 == fake.round()).sum(), (1 == (tr + vr).round()).sum())
print(0, (0 == fake.round()).sum(), (0 == (tr + vr).round()).sum())

5 tensor(851505, device='cuda:0') 226310
4 tensor(3571705, device='cuda:0') 348971
3 tensor(3212831, device='cuda:0') 261197
2 tensor(925470, device='cuda:0') 107557
1 tensor(87883, device='cuda:0') 56174
0 tensor(13434032, device='cuda:0') 21384031


In [44]:
np.save('./fake', fake.cpu().detach().numpy())

In [45]:
np.unique(tr + vr)

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [46]:
fake = fake.cpu().detach().int().numpy()

In [47]:
np.unique(fake)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [48]:
get_sparsity(fake)

39.984417608102845

In [49]:
get_sparsity(tr+vr)

4.468362562231285

In [50]:
np.clip(fake, 0, 5, out=fake)

array([[4, 3, 0, ..., 0, 4, 4],
       [0, 0, 0, ..., 0, 5, 0],
       [4, 0, 3, ..., 4, 5, 4],
       ...,
       [4, 3, 3, ..., 0, 3, 0],
       [0, 3, 0, ..., 3, 4, 4],
       [5, 0, 3, ..., 0, 4, 4]])

In [51]:
np.unique(fake)

array([0, 1, 2, 3, 4, 5])

In [52]:
np.save('./fake', fake)

In [104]:
rand_ix = np.random.randint(0, fake.shape[0], 100)

In [105]:
adding_fake = fake[rand_ix]

In [106]:
adding_fake

array([[4, 0, 0, ..., 4, 0, 4],
       [0, 3, 3, ..., 5, 0, 0],
       [4, 0, 3, ..., 0, 0, 4],
       ...,
       [0, 4, 0, ..., 4, 0, 0],
       [4, 3, 3, ..., 4, 5, 0],
       [4, 3, 3, ..., 4, 0, 4]])

In [107]:
np.unique(adding_fake[:,0])

array([0, 4, 5])

In [108]:
np.unique(fake[:,3])

array([0, 2, 3, 4])

In [109]:
np.unique(tr[:,10])

array([0., 1., 2., 3., 4., 5.])

In [110]:
get_sparsity(tr)

6.779468289293665

In [111]:
tr_orig, vr_1 = loadData('./ml-1m/ratings.dat', delimiter='::', seed=seed,  transpose=False, valfrac=0.1)

reading data...
data read in 4.607970714569092 seconds
loaded dense data matrix


In [112]:
tr = np.append(tr_orig, adding_fake, axis=0)

In [113]:
get_sparsity(tr)

4.606681479632465

In [114]:
np.unique(adding_fake[:,10])

array([0, 3, 4, 5])

In [115]:
np.save('./fake', fake)

In [116]:
to_concat = np.load('./fake.npy')

In [117]:
to_concat.astype(int)

array([[4, 3, 0, ..., 0, 4, 4],
       [0, 0, 0, ..., 0, 5, 0],
       [4, 0, 3, ..., 4, 5, 4],
       ...,
       [4, 3, 3, ..., 0, 3, 0],
       [0, 3, 0, ..., 3, 4, 4],
       [5, 0, 3, ..., 0, 4, 4]])

In [118]:
# # seed = 47

# # load data
# tr_1, vr_1 = loadData('./ml-1m/ratings.dat', delimiter='::', seed=seed,
#  transpose=True, valfrac=0.1)

# tm_1 = np.greater(tr_1, 1e-12).astype('float32')  # masks indicating non-zero entries
# vm_1 = np.greater(vr_1, 1e-12).astype('float32')

In [119]:
# (vr_1 == vr).all()

In [120]:
# (tr_1 == tr).all()

In [121]:
# tr_2, vr_2 = loadData('./ml-1m/ratings.dat', delimiter='::',
#                   seed=seed, transpose=False, valfrac=0.1)

In [122]:
# tr_2.shape

In [123]:
# !pip install nbimporter

In [124]:
import nbimporter 
import matrix_factorization

In [128]:
tr.shape, tr_orig.shape

((6140, 3706), (6040, 3706))

In [129]:
get_sparsity(tr)

4.606681479632465

In [130]:
get_sparsity(tr_orig)

4.021525859265269

In [125]:
MF_SGD = matrix_factorization.ExplicitMF(tr, 40, learning='sgd', verbose=True)
# iter_array = [1, 2, 5, 10, 25, 50, 100, 200]

iter_array = [40]
MF_SGD.calculate_learning_curve(iter_array, vr, learning_rate=0.001)

Iteration: 20
	current iteration: 10
	current iteration: 20
Train mse: 0.7288213535627215
Test mse: 0.8133318867936199


In [126]:
tr.shape, tr_orig.shape

((6140, 3706), (6040, 3706))

In [127]:
MF_SGD = matrix_factorization.ExplicitMF(tr_orig, 40, learning='sgd', verbose=True)
# iter_array = [1, 2, 5, 10, 25, 50, 100, 200]

iter_array = [20]
MF_SGD.calculate_learning_curve(iter_array, vr, learning_rate=0.001)

Iteration: 20
	current iteration: 10
	current iteration: 20
Train mse: 0.8135750792748816
Test mse: 0.8141703075881768


In [77]:
tr.shape

(6240, 3706)

In [78]:
tr_2 = np.append(tr, to_concat[:2000,:], axis=0)

In [79]:
MF_SGD = matrix_factorization.ExplicitMF(tr_2, 40, learning='sgd', verbose=True)
# iter_array = [1, 2, 5, 10, 25, 50, 100, 200]

iter_array = [10]
MF_SGD.calculate_learning_curve(iter_array, vr, learning_rate=0.001)

Iteration: 10
	current iteration: 10
Train mse: 0.3203302278807397
Test mse: 0.8314406623619328


In [80]:
tm = np.greater(tr, 1e-12).astype('float32')  # masks indicating non-zero entries
tm_orig = np.greater(tr_orig, 1e-12).astype('float32')
vm = np.greater(vr, 1e-12).astype('float32')

n_m = tr.shape[0]  # number of movies
n_u = tr.shape[1]  # number of users (may be switched depending on 'transpose' in loadData)

# Set hyper-parameters
n_hid = 500
# lambda_2 = float(sys.argv[1]) if len(sys.argv) > 1 else 60.
# lambda_s = float(sys.argv[2]) if len(sys.argv) > 2 else 0.013
n_layers = 2
output_every = 50  # evaluate performance on test set; breaks l-bfgs loop
n_epoch = n_layers * 10 * output_every
verbose_bfgs = True
use_gpu = True

In [81]:
if not use_gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    
# Input placeholders
R = tf.placeholder("float", [None, n_u])

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
# define network functions
def kernel(u, v):
    """
    Sparsifying kernel function

    :param u: input vectors [n_in, 1, n_dim]
    :param v: output vectors [1, n_hid, n_dim]
    :return: input to output connection matrix
    """
    dist = tf.norm(u - v, ord=2, axis=2)
    hat = tf.maximum(0., 1. - dist**2)
    return hat


def kernel_layer(x, n_hid=500, n_dim=5, activation=tf.nn.sigmoid, name=''):
    """
    a kernel sparsified layer

    :param x: input [batch, channels]
    :param n_hid: number of hidden units
    :param n_dim: number of dimensions to embed for kernelization
    :param activation: output activation
    :param name: layer name for scoping
    :return: layer output, regularization term
    """

    # define variables
    with tf.variable_scope(name):
        W = tf.get_variable('W', [x.shape[1], n_hid])
        n_in = x.get_shape().as_list()[1]
        u = tf.get_variable('u', initializer=tf.random.truncated_normal([n_in, 1, n_dim], 0., 1e-3))
        v = tf.get_variable('v', initializer=tf.random.truncated_normal([1, n_hid, n_dim], 0., 1e-3))
        b = tf.get_variable('b', [n_hid])

    # compute sparsifying kernel
    # as u and v move further from each other for some given pair of neurons, their connection
    # decreases in strength and eventually goes to zero.
    w_hat = kernel(u, v)

    # compute regularization terms
#     sparse_reg = tf.contrib.layers.l2_regularizer(lambda_s)
    sparse_reg_term = tf.contrib.layers.apply_regularization(sparse_reg, [w_hat])

    l2_reg = tf.contrib.layers.l2_regularizer(lambda_2)
    l2_reg_term = tf.contrib.layers.apply_regularization(l2_reg, [W])

    # compute output
    W_eff = W * w_hat
    y = tf.matmul(x, W_eff) + b
    y = activation(y)
    return y, sparse_reg_term + l2_reg_term


# Instantiate network
y = R
reg_losses = None
for i in range(n_layers):
    y, reg_loss = kernel_layer(y, n_hid, name=str(i))
    reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss
prediction, reg_loss = kernel_layer(y, n_u, activation=tf.identity, name='out')
reg_losses = reg_losses + reg_loss

# Compute loss (symbolic)
diff = tm*(R - prediction)
sqE = tf.nn.l2_loss(diff)
loss = sqE + reg_losses

# Instantiate L-BFGS Optimizer
optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, options={'maxiter': output_every,
                                                                  'disp': verbose_bfgs,
                                                                  'maxcor': 10},
                                                   method='L-BFGS-B')

In [ ]:
# Training and validation loop
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(int(n_epoch / output_every)):
        optimizer.minimize(sess, feed_dict={R: tr}) #do maxiter optimization steps
        pre = sess.run(prediction, feed_dict={R: tr}) #predict ratings

        error = (vm * (np.clip(pre, 1., 5.) - vr) ** 2).sum() / vm.sum() #compute validation error
        error_train = (tm * (np.clip(pre, 1., 5.) - tr) ** 2).sum() / tm.sum() #compute train error
        error_train_orig = (tm_orig * (np.clip(pre, 1., 5.) - tr_orig) ** 2).sum() / tm_orig.sum() #compute train error

        print('.-^-._' * 12)
        print('epoch:', i, 'validation rmse:', np.sqrt(error), 'train rmse:', np.sqrt(error_train), 'train orig rmse:', np.sqrt(error_train_orig))
        print('.-^-._' * 12)

    with open('summary_ml1m.txt', 'a') as file:
        for a in sys.argv[1:]:
            file.write(a + ' ')
        file.write(str(np.sqrt(error)) + ' ' + str(np.sqrt(error_train), + ' ' + str(np.sqrt(error_train))
                   + ' ' + str(seed) + '\n')
        file.close()